In [2]:
import timm
import torch
from timm.data.transforms_factory import create_transform
from timm.data import ImageDataset
from timm.optim import create_optimizer_v2 as create_optimizer
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader

In [ ]:
create_optimizer(model, )

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [6]:
import warnings
warnings.filterwarnings('ignore', '.*interpolation.*', )

model = timm.create_model('resnet50', pretrained=True, num_classes=1081)

dataset = ImageDataset(r"C:\Users\lulu5\Documents\echantillon_2", 
                       transform=create_transform(224, is_training=True))
loader = DataLoader(dataset, batch_size=100, shuffle=True, num_workers=16)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = create_optimizer(model, 'sgd', lr=0.01, momentum=0.9, weight_decay=1e-4)
#optimizer = create_optimizer.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = CrossEntropyLoss()

C:\Users\lulu5\anaconda3\lib\site-packages\torch\utils\data\dataloader.py:478: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
scheduler = MultiStepLR(optimizer, milestones=[30, 40], gamma=0.1)

In [9]:

for epoch in range(50):
    print(epoch)
    running_loss = 0
    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss.item()
    print('Loss:', running_loss/len(loader))
    scheduler.step()

0


KeyboardInterrupt: 